In [1]:
import librosa
import librosa.display
import os
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import datetime as dt
from keras import layers
from keras import models
from keras import optimizers
from sklearn.model_selection import train_test_split

os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

/usr/local/lib/python3.6/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)typ

In [2]:
# Set TF random seed to improve reproducibility
tf.set_random_seed(1234)
random.seed(100)

In [3]:
# define parameters
batch_size = 32
minimum_len = 128
epochs = 200
loss_function = 'categorical_crossentropy'
activation_function = 'softmax'

### Getting file names

In [4]:
#currdir= os.getcwd()
rootdir = '/home/taejoon/PhysioNetChallenge'
input_directory = os.path.join(rootdir, 'Training_WFDB')
mel_name = 'Mel_data_20200402_128' 
mel_directory = os.path.join(rootdir, mel_name)
results_directory = os.path.join(rootdir, 'results')
#save_directory = os.path.join(currdir, '')
if not os.path.isdir(input_directory):
        os.mkdir(input_directory)
if not os.path.isdir(mel_directory):
        os.mkdir(mel_directory)
if not os.path.isdir(results_directory):
        os.mkdir(results_directory)                       

In [5]:
# Find files
input_files = []
for f in os.listdir(input_directory):
    if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
        input_files.append(f)

In [6]:
input_file_names = sorted(input_files)
input_file_names

['A0001.mat',
 'A0002.mat',
 'A0003.mat',
 'A0004.mat',
 'A0005.mat',
 'A0006.mat',
 'A0007.mat',
 'A0008.mat',
 'A0009.mat',
 'A0010.mat',
 'A0011.mat',
 'A0012.mat',
 'A0013.mat',
 'A0014.mat',
 'A0015.mat',
 'A0016.mat',
 'A0017.mat',
 'A0018.mat',
 'A0019.mat',
 'A0020.mat',
 'A0021.mat',
 'A0022.mat',
 'A0023.mat',
 'A0024.mat',
 'A0025.mat',
 'A0026.mat',
 'A0027.mat',
 'A0028.mat',
 'A0029.mat',
 'A0030.mat',
 'A0031.mat',
 'A0032.mat',
 'A0033.mat',
 'A0034.mat',
 'A0035.mat',
 'A0036.mat',
 'A0037.mat',
 'A0038.mat',
 'A0039.mat',
 'A0040.mat',
 'A0041.mat',
 'A0042.mat',
 'A0043.mat',
 'A0044.mat',
 'A0045.mat',
 'A0046.mat',
 'A0047.mat',
 'A0048.mat',
 'A0049.mat',
 'A0050.mat',
 'A0051.mat',
 'A0052.mat',
 'A0053.mat',
 'A0054.mat',
 'A0055.mat',
 'A0056.mat',
 'A0057.mat',
 'A0058.mat',
 'A0059.mat',
 'A0060.mat',
 'A0061.mat',
 'A0062.mat',
 'A0063.mat',
 'A0064.mat',
 'A0065.mat',
 'A0066.mat',
 'A0067.mat',
 'A0068.mat',
 'A0069.mat',
 'A0070.mat',
 'A0071.mat',
 'A007

### Code for extracting only single-label subjects

In [7]:
# Find unique number of classes  
def get_unique_classes(input_directory,files):

    unique_classes=set()
    for f in files:
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    for c in tmp:
                        unique_classes.add(c.strip())

    return sorted(unique_classes)

unique_classes = get_unique_classes(input_directory, input_files)
# Creating one-hot vector for Y
# num = np.unique(classes, axis=0)
class2index = {}
for a, b in enumerate(unique_classes):
    class2index[b] = a
#class2index

def one_hot_encoding(one_hot_vector,y, class2index):

       ind=class2index[y]
       one_hot_vector[ind]=1
       return one_hot_vector

In [8]:
class2index

{'AF': 0,
 'I-AVB': 1,
 'LBBB': 2,
 'Normal': 3,
 'PAC': 4,
 'PVC': 5,
 'RBBB': 6,
 'STD': 7,
 'STE': 8}

In [9]:
# Search for multi-label subjects
def searching_overlap(input_directory,class2index, input_file_names):
    multiclasses=[]
    multisubjects=[]
    number = []
    for file in input_file_names:
        f=file
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    if len(tmp)>1:
                        one_hot_vector = [0]*(len(class2index))
                        for c in tmp:
                            one_hot_vector = one_hot_encoding(one_hot_vector, c.strip(), class2index)
                        multiclasses.append(one_hot_vector)
                        multisubjects.append(g)
                        number.append(len(tmp))
    return multisubjects, multiclasses, number

In [10]:
classes_orig= [x.replace('.mat', '.hea') for x in input_file_names] # total subjects
classes_multi, _, _ = searching_overlap(input_directory,class2index, input_file_names)
classes_single = [x for x in classes_orig if x not in classes_multi]
classes_single = [x.replace('.hea', '.mat') for x in classes_single]

In [11]:
np.shape(classes_single)

(6401,)

In [12]:
classes_single

['A0001.mat',
 'A0002.mat',
 'A0003.mat',
 'A0004.mat',
 'A0005.mat',
 'A0006.mat',
 'A0007.mat',
 'A0008.mat',
 'A0009.mat',
 'A0010.mat',
 'A0011.mat',
 'A0012.mat',
 'A0013.mat',
 'A0014.mat',
 'A0015.mat',
 'A0016.mat',
 'A0017.mat',
 'A0018.mat',
 'A0019.mat',
 'A0020.mat',
 'A0021.mat',
 'A0022.mat',
 'A0023.mat',
 'A0024.mat',
 'A0025.mat',
 'A0026.mat',
 'A0027.mat',
 'A0028.mat',
 'A0029.mat',
 'A0030.mat',
 'A0031.mat',
 'A0032.mat',
 'A0033.mat',
 'A0034.mat',
 'A0035.mat',
 'A0036.mat',
 'A0037.mat',
 'A0038.mat',
 'A0039.mat',
 'A0040.mat',
 'A0041.mat',
 'A0042.mat',
 'A0044.mat',
 'A0045.mat',
 'A0046.mat',
 'A0047.mat',
 'A0048.mat',
 'A0049.mat',
 'A0050.mat',
 'A0051.mat',
 'A0052.mat',
 'A0053.mat',
 'A0054.mat',
 'A0055.mat',
 'A0056.mat',
 'A0057.mat',
 'A0058.mat',
 'A0059.mat',
 'A0060.mat',
 'A0061.mat',
 'A0062.mat',
 'A0063.mat',
 'A0064.mat',
 'A0065.mat',
 'A0066.mat',
 'A0067.mat',
 'A0068.mat',
 'A0069.mat',
 'A0070.mat',
 'A0071.mat',
 'A0072.mat',
 'A007

In [13]:
# double-checking if classes_single have single-label
a, b, c  = searching_overlap(input_directory,class2index,classes_single)
a

[]

In [14]:
# we can safely use classes_single as input_file_names
input_file_names = classes_single
random.shuffle(input_file_names)
np.shape(input_file_names)

(6401,)

### Shuffle and divide files into train/eval/test

In [15]:
data, data_test = train_test_split(input_file_names, test_size = 0.2, train_size = 0.8, shuffle=True)
data_train, data_val = train_test_split(data, test_size = 0.25, train_size = 0.75, shuffle=True)

In [16]:
print(np.shape(data_train), np.shape(data_val), np.shape(data_test))

(3840,) (1280,) (1281,)


### CNN Model

In [17]:
from keras.applications.densenet import DenseNet169
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model 

input_tensor = Input(shape=(128, 128, 12))
base_model = DenseNet169(input_tensor=input_tensor, weights=None, include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
pred = Dense(9, activation=activation_function)(x)

model = Model(inputs=base_model.input, outputs=pred)

In [18]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 12) 0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 134, 134, 12) 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 64, 64, 64)   37632       zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 64, 64, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [19]:
def block_feature(sequence_en, minimum_len): 
    new_en = []
    if len(sequence_en) > minimum_len:  # 길이가 minimum보다 긴 경우
        start = random.randint(0,len(sequence_en)-minimum_len)    
        new_en = sequence_en[start:start+minimum_len]
    elif len(sequence_en) == minimum_len: # 길이가 minimum
        new_en = sequence_en
    else: 
        assert len(sequence_en) <= minimum_len
    return new_en

In [20]:
def exploratory_look(input_directory,file, class2index):
    
    classes = []

    f = file
    g = f.replace('.mat','.hea')
    input_file = os.path.join(input_directory,g)
    with open(input_file,'r') as f:
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                print(tmp, len(tmp))
    return tmp            

for file in input_file_names:
    tmp = exploratory_look(input_directory, file, class2index)

['Normal\n'] 1
['STD\n'] 1
['I-AVB\n'] 1
['I-AVB\n'] 1
['RBBB\n'] 1
['AF\n'] 1
['AF\n'] 1
['Normal\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['I-AVB\n'] 1
['AF\n'] 1
['PAC\n'] 1
['PAC\n'] 1
['STD\n'] 1
['RBBB\n'] 1
['PVC\n'] 1
['STE\n'] 1
['PVC\n'] 1
['RBBB\n'] 1
['AF\n'] 1
['PAC\n'] 1
['STD\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['I-AVB\n'] 1
['AF\n'] 1
['PAC\n'] 1
['STD\n'] 1
['Normal\n'] 1
['AF\n'] 1
['STE\n'] 1
['I-AVB\n'] 1
['STD\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['STD\n'] 1
['LBBB\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['PAC\n'] 1
['Normal\n'] 1
['I-AVB\n'] 1
['STD\n'] 1
['AF\n'] 1
['STD\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['PVC\n'] 1
['AF\n'] 1
['PAC\n'] 1
['PVC\n'] 1
['RBBB\n'] 1
['I-AVB\n'] 1
['LBBB\n'] 1
['PVC\n'] 1
['STD\n'] 1
['RBBB\n'] 1
['PVC\n'] 1
['PVC\n'] 1
['I-AVB\n'] 1
['RBBB\n'] 1
['PAC\n'] 1
['LBBB\n'] 1
['I-AVB\n'] 1
['RBBB\n'] 1
['STD\n'] 1
['AF\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['AF\n'] 1
['I-AVB\n'] 1
['STD\n'] 1
['STE\n'] 1
['Normal\n'] 1
['LBBB\n'] 1
['STD\n'] 1
['

['RBBB\n'] 1
['RBBB\n'] 1
['Normal\n'] 1
['I-AVB\n'] 1
['PAC\n'] 1
['RBBB\n'] 1
['I-AVB\n'] 1
['I-AVB\n'] 1
['Normal\n'] 1
['STE\n'] 1
['RBBB\n'] 1
['PAC\n'] 1
['PVC\n'] 1
['RBBB\n'] 1
['STD\n'] 1
['RBBB\n'] 1
['AF\n'] 1
['Normal\n'] 1
['I-AVB\n'] 1
['AF\n'] 1
['PVC\n'] 1
['PVC\n'] 1
['I-AVB\n'] 1
['Normal\n'] 1
['AF\n'] 1
['STE\n'] 1
['PVC\n'] 1
['Normal\n'] 1
['STD\n'] 1
['AF\n'] 1
['PAC\n'] 1
['AF\n'] 1
['PAC\n'] 1
['I-AVB\n'] 1
['PAC\n'] 1
['STE\n'] 1
['I-AVB\n'] 1
['PVC\n'] 1
['STD\n'] 1
['Normal\n'] 1
['Normal\n'] 1
['STD\n'] 1
['STD\n'] 1
['PAC\n'] 1
['AF\n'] 1
['PAC\n'] 1
['Normal\n'] 1
['PVC\n'] 1
['STD\n'] 1
['STD\n'] 1
['RBBB\n'] 1
['I-AVB\n'] 1
['PAC\n'] 1
['RBBB\n'] 1
['Normal\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['STD\n'] 1
['AF\n'] 1
['I-AVB\n'] 1
['STE\n'] 1
['RBBB\n'] 1
['STD\n'] 1
['Normal\n'] 1
['Normal\n'] 1
['RBBB\n'] 1
['STE\n'] 1
['PAC\n'] 1
['PAC\n'] 1
['PAC\n'] 1
['Normal\n'] 1
['RBBB\n'] 1
['AF\n'] 1
['AF\n'] 1
['RBBB\n'] 1
['Normal\n'] 1
['RBBB\n'] 1
['PAC\n'] 1
[

['AF\n'] 1
['AF\n'] 1
['STE\n'] 1
['I-AVB\n'] 1
['PVC\n'] 1
['STE\n'] 1
['PVC\n'] 1
['Normal\n'] 1
['STD\n'] 1
['AF\n'] 1
['I-AVB\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['Normal\n'] 1
['I-AVB\n'] 1
['STD\n'] 1
['AF\n'] 1
['Normal\n'] 1
['Normal\n'] 1
['STD\n'] 1
['AF\n'] 1
['PVC\n'] 1
['STE\n'] 1
['I-AVB\n'] 1
['AF\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['Normal\n'] 1
['PAC\n'] 1
['AF\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['STD\n'] 1
['Normal\n'] 1
['RBBB\n'] 1
['STD\n'] 1
['I-AVB\n'] 1
['RBBB\n'] 1
['STD\n'] 1
['PVC\n'] 1
['Normal\n'] 1
['STD\n'] 1
['RBBB\n'] 1
['STD\n'] 1
['RBBB\n'] 1
['AF\n'] 1
['RBBB\n'] 1
['AF\n'] 1
['PVC\n'] 1
['RBBB\n'] 1
['AF\n'] 1
['STD\n'] 1
['RBBB\n'] 1
['STE\n'] 1
['AF\n'] 1
['LBBB\n'] 1
['RBBB\n'] 1
['Normal\n'] 1
['I-AVB\n'] 1
['AF\n'] 1
['AF\n'] 1
['AF\n'] 1
['STE\n'] 1
['PVC\n'] 1
['AF\n'] 1
['AF\n'] 1
['I-AVB\n'] 1
['RBBB\n'] 1
['AF\n'] 1
['I-AVB\n'] 1
['STE\n'] 1
['Normal\n'] 1
['STE\n'] 1
['PVC\n'] 1
['STD\n'] 1
['AF\n'] 1
['Normal\n'] 1
['STD\n'] 1
['PAC\n

['RBBB\n'] 1
['RBBB\n'] 1
['PAC\n'] 1
['AF\n'] 1
['PVC\n'] 1
['RBBB\n'] 1
['PVC\n'] 1
['RBBB\n'] 1
['Normal\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['Normal\n'] 1
['PVC\n'] 1
['STD\n'] 1
['RBBB\n'] 1
['PVC\n'] 1
['RBBB\n'] 1
['Normal\n'] 1
['AF\n'] 1
['PAC\n'] 1
['Normal\n'] 1
['I-AVB\n'] 1
['Normal\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['PVC\n'] 1
['AF\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['AF\n'] 1
['AF\n'] 1
['RBBB\n'] 1
['Normal\n'] 1
['Normal\n'] 1
['Normal\n'] 1
['STD\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['PAC\n'] 1
['Normal\n'] 1
['Normal\n'] 1
['STD\n'] 1
['RBBB\n'] 1
['PVC\n'] 1
['PVC\n'] 1
['RBBB\n'] 1
['I-AVB\n'] 1
['PAC\n'] 1
['AF\n'] 1
['RBBB\n'] 1
['Normal\n'] 1
['RBBB\n'] 1
['AF\n'] 1
['RBBB\n'] 1
['PVC\n'] 1
['STD\n'] 1
['PVC\n'] 1
['STE\n'] 1
['STD\n'] 1
['PAC\n'] 1
['RBBB\n'] 1
['AF\n'] 1
['AF\n'] 1
['I-AVB\n'] 1
['RBBB\n'] 1
['RBBB\n'] 1
['AF\n'] 1
['AF\n'] 1
['LBBB\n'] 1
['STE\n'] 1
['RBBB\n'] 1
['STD\n'] 1
['AF\n'] 1
['STD\n'] 1
['RBBB\n'] 1
['STD\n'] 1
['PVC\n'] 1
['AF\n'] 1
['STD\n'] 1


In [21]:
# Get classes of sorted file names
def get_labels(input_directory,file, class2index):
    f = file
    g = f.replace('.mat','.hea')
    input_file = os.path.join(input_directory,g)
    with open(input_file,'r') as f:
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                one_hot_vector = [0]*(len(class2index))
                for c in tmp:
                    one_hot_vector = one_hot_encoding(one_hot_vector, c.strip(), class2index)
                
    return one_hot_vector

In [22]:
def randextract_mels(curr_step, batch_size, data, mel_directory, class2index, minimum_len): # step = 0, 1, 2, 3....
    mel_files = []
    classes = []
    start = batch_size*curr_step
    end = batch_size*(curr_step+1)
    curr_file_indices = data[start:end]
    
    # 여기서 두 번째 셔플 
    random.shuffle(curr_file_indices)
    
    for file in curr_file_indices:
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        clip_file = block_feature(tmp_file, minimum_len)
        mel_files.append(clip_file)
        
        label = get_labels(input_directory, file, class2index)
        classes.append(label)
    return mel_files, classes

In [23]:
def randextract_mels2(data, mel_directory, class2index, minimum_len): # step = 0, 1, 2, 3....
    tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
    clip_file = block_feature(tmp_file, minimum_len)
    mel_file = clip_file
        
    label = get_labels(input_directory, file, class2index)
    classes = label
    return mel_files, classes

In [24]:
model.compile(loss=loss_function,
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [25]:
def train(data_train, mel_directory, batch_size, class2index, minimum_len): 
    
    loss=[]
    acc = []
    # 여기서 첫 번째로 셔플하고 
    random.shuffle(data_train)
    total_steps = int(np.ceil(len(data_train)/batch_size))
    for curr_step in range(total_steps):
        batch_mels, batch_labels = randextract_mels(curr_step, batch_size, data_train, mel_directory, class2index, minimum_len)
        batch_mels = np.asarray(batch_mels)
        batch_labels = np.asarray(np.squeeze(batch_labels))
#         print(np.shape(batch_labels))
        train_tmp = model.train_on_batch(batch_mels, batch_labels)
        loss.append(train_tmp[0])
        acc.append(train_tmp[1])

    loss = np.mean(np.array(loss))
    acc = np.mean(np.array(acc))
    return loss, acc

In [26]:
def test(data, mel_directory, input_directory, class2index, minimum_len, model): 
    # 1. 파일 읽어오기
    # 2. 128 길이의 block들로 쪼개기 (끄트머리 버리기-> np.floor)
    # 3. 각 block -> model.predict -> logit 가져오기 
    # 4. Block들간 평균내서 1 or 0 부여
    # 5. 실제 label과 비교 

    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    for i, file in enumerate(data):
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        logits=[]
        for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
            start = block*minimum_len
            end = (block+1)*minimum_len
            curr_block = tmp_file[start:end]
            curr_block = np.expand_dims(curr_block,0)

            logit = model.predict(curr_block) # sigmoid 거쳐서 나온 것
            logits.append(logit)

        average = np.mean(np.squeeze(logits), axis=0)
        scores.append(average)
#         average[average>=0.5] = 1
#         average[average<0.5] = 0
        pred_ans = np.argmax(average)
    
        if average[average<0]:
            print('Error: output of sigmoid should be between 0 and 1')
#         predicted_labels.append(average)    
        predicted_labels.append(pred_ans)
        true_label = get_labels(input_directory, file, class2index)
        true_ans = np.argmax(true_label)
#         if np.array_equal(average, true_label): # predicted와 실제 label이 아예 같아야만 정답으로 인정
        if int(true_ans) == int(pred_ans):
            accuracy[i] = 1

    avg_acc = np.mean(accuracy)
    return avg_acc, scores, predicted_labels

In [28]:
def test2(data, mel_directory, input_directory, class2index, minimum_len, model): # this function uses test_on_batch
    # 1. 파일 읽어오기
    # 2. 128 길이의 block들로 쪼개기 (끄트머리 버리기-> np.floor)
    # 3. 각 block -> model.predict -> logit 가져오기 
    # 4. Block들간 평균내서 1 or 0 부여
    # 5. 실제 label과 비교 

    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    total_loss=[]
    total_acc=[]
    for i, file in enumerate(data):
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        loss=[]
        acc=[]
        y_true = get_labels(input_directory, file, class2index)
        y_true = np.transpose(np.expand_dims(y_true,1))
        for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
            start = block*minimum_len
            end = (block+1)*minimum_len
            curr_block = tmp_file[start:end]
            curr_block = np.expand_dims(curr_block,0)


            test_tmp = model.test_on_batch(curr_block, y_true)
            loss.append(test_tmp[0])
            acc.append(test_tmp[1])
            
        total_loss.append(np.mean(np.array(loss)))
        total_acc.append(np.mean(np.array(acc)))
        

    avg_acc = np.mean(np.array(total_acc))
    avg_loss = np.mean(np.array(total_loss))
    return avg_loss, avg_acc
# avg_acc, scores, predicted_labels

In [ ]:
def test3(data, mel_directory, input_directory, class2index, minimum_len, model): # this function uses test_on_batch
    # 1. 파일 읽어오기
    # 2. 128 길이의 block들로 쪼개기 (끄트머리 버리기-> np.floor)
    # 3. 각 block -> model.predict -> logit 가져오기 
    # 4. Block들간 평균내서 1 or 0 부여
    # 5. 실제 label과 비교 

    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    total_loss=[]
    total_acc=[]
    for i, file in enumerate(data):
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        loss=[]
        acc=[]
        y_true = get_labels(input_directory, file, class2index)
        y_true = np.transpose(np.expand_dims(y_true,1))
#         for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
#             start = block*minimum_len
#             end = (block+1)*minimum_len
        curr_block = tmp_file[:minimum_len]
        curr_block = np.expand_dims(curr_block,0)

        test_tmp = model.predict_on_batch(curr_block)
#         test_tmp = model.test_on_batch(curr_block, y_true)
#         loss.append(test_tmp[0])
#         acc.append(test_tmp[1])
#         total_loss.append(np.mean(np.array(loss)))
#         total_acc.append(np.mean(np.array(acc)))
        scores.append(test_tmp)
        pred_ans = np.argmax(test_tmp)
        predicted_labels.append(pred_ans)
#     avg_acc = np.mean(np.array(total_acc))
#     avg_loss = np.mean(np.array(total_loss))
    return predicted_labels, scores

In [29]:
val_acc_sum=[]
train_loss_sum=[]
train_acc_sum=[]
val_loss_sum=[] 
for num_epoch in range(epochs):
    train_loss, train_acc = train(data_train, mel_directory, batch_size, class2index, minimum_len)
    print('\nEpoch',num_epoch+1,'train_loss:',f'{train_loss:.3f}','train_acc:',f'{train_acc:.3f}',"\t", dt.datetime.now())
    model_output = "ecg_mel_E%02dL%.2f" % (num_epoch, train_loss)
    save_name = os.path.join(results_directory, model_output)
    
    val_loss, val_acc = test2(data_val, mel_directory, input_directory, class2index, minimum_len, model)
    print('\nValidation', num_epoch+1,'valid_loss:',f'{val_loss:.3f}','valid_acc:',f'{val_acc:.3f}',"\t", dt.datetime.now()) 
#     val_acc, _, _ = test(data_val, mel_directory, input_directory, class2index, minimum_len, model)
#     print('\nValidation', num_epoch+1,'valid_acc:',f'{val_acc:.3f}',"\t", dt.datetime.now()) 
    val_acc_sum.append(val_acc)
    val_loss_sum.append(val_loss)
    train_loss_sum.append(train_loss)
    train_acc_sum.append(train_acc)

#     model.save(save_name)


Epoch 1 train_loss: 1.437 train_acc: 0.518 	 2020-04-17 08:33:50.619887

Validation 1 valid_loss: 3.684 valid_acc: 0.091 	 2020-04-17 08:35:29.962804

Epoch 2 train_loss: 1.282 train_acc: 0.568 	 2020-04-17 08:35:48.508714

Validation 2 valid_loss: 7.745 valid_acc: 0.235 	 2020-04-17 08:37:26.255025

Epoch 3 train_loss: 1.182 train_acc: 0.607 	 2020-04-17 08:37:44.826105

Validation 3 valid_loss: 4.739 valid_acc: 0.195 	 2020-04-17 08:39:23.046255

Epoch 4 train_loss: 1.120 train_acc: 0.614 	 2020-04-17 08:39:41.617432

Validation 4 valid_loss: 2.850 valid_acc: 0.244 	 2020-04-17 08:41:19.763364

Epoch 5 train_loss: 1.036 train_acc: 0.657 	 2020-04-17 08:41:38.155193

Validation 5 valid_loss: 1.346 valid_acc: 0.568 	 2020-04-17 08:43:16.308412

Epoch 6 train_loss: 0.977 train_acc: 0.676 	 2020-04-17 08:43:34.897357

Validation 6 valid_loss: 1.706 valid_acc: 0.400 	 2020-04-17 08:45:13.040205

Epoch 7 train_loss: 0.953 train_acc: 0.685 	 2020-04-17 08:45:31.420406

Validation 7 valid_l


Epoch 55 train_loss: 0.346 train_acc: 0.878 	 2020-04-17 10:18:51.001696

Validation 55 valid_loss: 1.586 valid_acc: 0.568 	 2020-04-17 10:20:29.214874

Epoch 56 train_loss: 0.324 train_acc: 0.889 	 2020-04-17 10:20:47.801087

Validation 56 valid_loss: 2.144 valid_acc: 0.584 	 2020-04-17 10:22:25.947609

Epoch 57 train_loss: 0.351 train_acc: 0.875 	 2020-04-17 10:22:44.537851

Validation 57 valid_loss: 3.213 valid_acc: 0.424 	 2020-04-17 10:24:22.761553

Epoch 58 train_loss: 0.322 train_acc: 0.885 	 2020-04-17 10:24:41.165289

Validation 58 valid_loss: 3.793 valid_acc: 0.460 	 2020-04-17 10:26:19.327083

Epoch 59 train_loss: 0.307 train_acc: 0.890 	 2020-04-17 10:26:37.949996

Validation 59 valid_loss: 2.402 valid_acc: 0.480 	 2020-04-17 10:28:16.029776

Epoch 60 train_loss: 0.320 train_acc: 0.887 	 2020-04-17 10:28:34.415462

Validation 60 valid_loss: 2.348 valid_acc: 0.554 	 2020-04-17 10:30:12.338780

Epoch 61 train_loss: 0.321 train_acc: 0.881 	 2020-04-17 10:30:30.886887

Validat


Validation 108 valid_loss: 1.700 valid_acc: 0.612 	 2020-04-17 12:03:33.338011

Epoch 109 train_loss: 0.152 train_acc: 0.949 	 2020-04-17 12:03:51.908669

Validation 109 valid_loss: 1.771 valid_acc: 0.626 	 2020-04-17 12:05:30.048572

Epoch 110 train_loss: 0.150 train_acc: 0.949 	 2020-04-17 12:05:48.652459

Validation 110 valid_loss: 3.537 valid_acc: 0.461 	 2020-04-17 12:07:26.626170

Epoch 111 train_loss: 0.169 train_acc: 0.944 	 2020-04-17 12:07:45.212643

Validation 111 valid_loss: 5.738 valid_acc: 0.400 	 2020-04-17 12:09:23.309530

Epoch 112 train_loss: 0.152 train_acc: 0.948 	 2020-04-17 12:09:41.897769

Validation 112 valid_loss: 4.260 valid_acc: 0.419 	 2020-04-17 12:11:19.907693

Epoch 113 train_loss: 0.153 train_acc: 0.950 	 2020-04-17 12:11:38.496817

Validation 113 valid_loss: 7.128 valid_acc: 0.318 	 2020-04-17 12:13:16.603768

Epoch 114 train_loss: 0.141 train_acc: 0.954 	 2020-04-17 12:13:35.221981

Validation 114 valid_loss: 4.482 valid_acc: 0.440 	 2020-04-17 12:15:


Validation 161 valid_loss: 3.021 valid_acc: 0.562 	 2020-04-17 13:46:38.995970

Epoch 162 train_loss: 0.092 train_acc: 0.969 	 2020-04-17 13:46:57.575265

Validation 162 valid_loss: 4.074 valid_acc: 0.448 	 2020-04-17 13:48:36.084435

Epoch 163 train_loss: 0.113 train_acc: 0.961 	 2020-04-17 13:48:54.678523

Validation 163 valid_loss: 2.566 valid_acc: 0.633 	 2020-04-17 13:50:32.838683

Epoch 164 train_loss: 0.095 train_acc: 0.969 	 2020-04-17 13:50:51.257730

Validation 164 valid_loss: 2.587 valid_acc: 0.598 	 2020-04-17 13:52:29.330826

Epoch 165 train_loss: 0.066 train_acc: 0.976 	 2020-04-17 13:52:47.737746

Validation 165 valid_loss: 5.040 valid_acc: 0.356 	 2020-04-17 13:54:25.979599

Epoch 166 train_loss: 0.089 train_acc: 0.971 	 2020-04-17 13:54:44.589977

Validation 166 valid_loss: 4.596 valid_acc: 0.378 	 2020-04-17 13:56:22.729892

Epoch 167 train_loss: 0.102 train_acc: 0.966 	 2020-04-17 13:56:41.401129

Validation 167 valid_loss: 3.840 valid_acc: 0.514 	 2020-04-17 13:58:

In [1]:
x_range = range(epochs)
plt.plot(x_range, train_acc_sum, 'bo', label='Training Accuracy')
plt.plot(x_range, val_acc_sum, 'ro', label='Validation Accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(x_range, train_loss_sum, 'bo', label='Training Loss')
plt.plot(x_range, val_loss_sum, 'ro', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

NameError: name 'epochs' is not defined

### Now, test with test data

In [ ]:
# Scoring:

# For each 12-lead ECG recording, your algorithm must identify a set of one or more classes as well as a probability or confidence score for each class. 

# Example: 
# #Record ID
#  AF, I-AVB, LBBB, Normal, RBBB, PAC,  PVC,  STD, STE
#   1,     1,    0,      0,    0,   0,   0,     0,   0
# 0.9,   0.6,  0.2,   0.05,  0.2, 0.35, 0.35, 0.1, 0.1

Since some recordings may have multiple labels, we normalize their contributions to these scoring functions so that each recording, not each class, makes an equal contribution. For example, if a recording has six classes, and your classifier identifies three labels correctly, identifies one label incorrectly, and misses two labels, then we increment the true positives by 3/6, false positives by 1/6, and false negatives by 2/6 for both scoring functions.

In [ ]:
# def test(data_test, mel_directory, class2index, minimum_len, model): 
    
#     metrics = []
#     batch_mels, batch_labels = randextract_mels_val(0, len(data_test)-1, data_val, mel_directory, class2index, minimum_len)
#     # although rendextract_mels_val, you can use the same function fpr test
#     batch_mels = np.asarray(batch_mels)
#     batch_labels = np.asarray(np.squeeze(batch_labels))
#     test_loss_tmp = model.test_on_batch(batch_mels, batch_labels)
#     loss = test_loss_tmp[0]
#     acc = test_loss_tmp[1]
# #     metrics.append(test_loss_tmp)

# #     metrics = np.mean(np.array(metrics))
#     return loss, acc

In [ ]:
# class2index = {'AF': 0,
#                'I-AVB': 1,
#                'LBBB': 2,
#                'Normal': 3,
#                'PAC': 4,
#                'PVC': 5,
#                'RBBB': 6,
#                'STD': 7,
#                'STE': 8}
class2index = {}
for a, b in enumerate(unique_classes):
    class2index[b] = a
classes = class2index.keys()
classes


In [ ]:
class2index[1]

In [ ]:
def save_challenge_predictions(data, scores, predicted_labels, classes, csv_directory):
    for i, filename in enumerate(data):
        curr_score = scores[i]
        curr_score=curr_score[0]
        curr_score = np.asarray(curr_score)
        curr_label = predicted_labels[i]
        new_file = filename.replace('.mat','.csv')
        output_file = os.path.join(csv_directory,new_file)
        recording = new_file.replace('.csv', '')
        # Include the filename as the recording number
        recording_string = '#{}'.format(recording)
        class_string = ','.join(classes)
        label_string = ','.join(str(i) for i in curr_label)
        score_string = ','.join(str(i) for i in curr_score)

        with open(output_file, 'w') as f:
            f.write(recording_string + '\n' + class_string + '\n' + label_string + '\n' + score_string + '\n')

In [ ]:
# load model
#  test(data, epochs, minimum_len,flag='val', ct=None): # valid에서 이 함수 부를 떄는 ct 인자로 넣어주기
rootdir = '/home/taejoon/PhysioNetChallenge'
csv_directory = os.path.join(rootdir, 'results', 'csvfiles')
if not os.path.isdir(csv_directory):
        os.mkdir(csv_directory)    
# model = tf.keras.models.load_model(os.path.join(results_directory, "ecg_mel_E200*"))
predicted_labels, scores = test3(data_test, mel_directory, input_directory, class2index,minimum_len, model)


In [ ]:
predicted_onehot = []
for x in predicted_labels:
    one_hot_vector = [0]*(len(class2index))
    one_hot_vector[x] = 1
    predicted_onehot.append(one_hot_vector)
save_challenge_predictions(data_test, scores, predicted_onehot, classes, csv_directory)
#     print('\nTest result: loss:',f'{test_metrics[0]:.3f}','accuracy:', f'{test_metrics[1]:.3f}',"\t", dt.datetime.now())


In [ ]:
# copy files because the scoring function requires same number of .hea*.mat files and test output files 
import shutil
label_dir = os.path.join(input_directory, 'test_labels')
if not os.path.isdir(label_dir):
        os.mkdir(label_dir)
for folder in data_test:
    shutil.copy(os.path.join(input_directory, folder), os.path.join(label_dir, folder)) # moves .mat file
    folder = folder.replace('.mat', '.hea')
    shutil.copy(os.path.join(input_directory, folder), os.path.join(label_dir, folder)) # moves .hea file

In [ ]:
from scipy.stats.mstats import gmean
gmean([0.319, 0.154])